# Data Imputation
- Temperature has the highest % of missing values at 47%
- Removed any counties that were missing values over all of the recorded dates
    - 744 counties from 3142 were removed (23.7%)
- Used SimpleImputer for categorical features
- Used KNN imputation for numeric features
- Imputed using non-missing values from columns

In [104]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import zipfile

In [53]:
data = pd.read_csv("/Users/Huey.ts/Desktop/STAT_390/data/data_final/us-covid-raw-data.csv")

In [54]:
# NULL Values
null = data.isnull().sum()
null[null != 0]

social_distancing_total_grade              122020
social_distancing_visitation_grade         324202
social_distancing_encounters_grade         122020
social_distancing_travel_distance_grade    122020
daily_state_test                           132248
precipitation                              256910
temperature                                352950
ventilator_capacity_ratio                   14340
icu_beds_ratio                              14340
Religious_congregation_ratio                  478
percent_insured                               239
deaths_per_100000                           20076
dtype: int64

In [55]:
# Percent % of NULL Values
null_per = data.isnull().mean()*100
null_per[null_per != 0]

social_distancing_total_grade              16.249011
social_distancing_visitation_grade         43.172938
social_distancing_encounters_grade         16.249011
social_distancing_travel_distance_grade    16.249011
daily_state_test                           17.611041
precipitation                              34.211879
temperature                                47.001217
ventilator_capacity_ratio                   1.909612
icu_beds_ratio                              1.909612
Religious_congregation_ratio                0.063654
percent_insured                             0.031827
deaths_per_100000                           2.673456
dtype: float64

In [56]:
null_features = [null_per[null_per != 0].index]

## Found and removed counties that were missing values over all of the recorded dates

In [57]:
null_dat = data.groupby("county_name", dropna=False).apply(lambda x: x.isnull().mean()*100)
null_dat

,date,county_fips,county_name,state_fips,state_name,covid_19_confirmed_cases,covid_19_deaths,social_distancing_total_grade,social_distancing_visitation_grade,social_distancing_encounters_grade,...,age_45_49,age_50_54,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_or_higher,immigrant_student_ratio
county_name,,,,,,,,,,,,,,,,,,,,,
Abbeville County,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.807531,100.000000,13.807531,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acadia Parish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.807531,13.807531,13.807531,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Accomack County,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.807531,13.807531,13.807531,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ada County,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.807531,13.807531,13.807531,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Adair County,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.807531,56.903766,13.807531,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yukon-Koyukuk Census Area,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.000000,100.000000,100.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yuma County,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.807531,56.903766,13.807531,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zapata County,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.807531,13.807531,13.807531,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
counties = []
for i in null_features:
    counties.append(null_dat.iloc[np.where(null_dat[i] == 100)[0]].index)

In [59]:
counties = np.concatenate(counties)
counties = np.unique(counties)

In [60]:
# Number of counties to remove
len(counties)

744

In [61]:
new_data = data.loc[~data["county_name"].isin(counties)].reset_index()
new_data

,index,date,county_fips,county_name,state_fips,state_name,covid_19_confirmed_cases,covid_19_deaths,social_distancing_total_grade,social_distancing_visitation_grade,...,age_45_49,age_50_54,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_or_higher,immigrant_student_ratio
0,239,01/22/20,1003,Baldwin County,1,Alabama,0.0,0.0,NaN,NaN,...,7,7,7,7,6,4,3,2,2,0.021048
1,240,01/23/20,1003,Baldwin County,1,Alabama,0.0,0.0,NaN,NaN,...,7,7,7,7,6,4,3,2,2,0.021048
2,241,01/24/20,1003,Baldwin County,1,Alabama,0.0,0.0,NaN,NaN,...,7,7,7,7,6,4,3,2,2,0.021048
3,242,01/25/20,1003,Baldwin County,1,Alabama,0.0,0.0,NaN,NaN,...,7,7,7,7,6,4,3,2,2,0.021048
4,243,01/26/20,1003,Baldwin County,1,Alabama,0.0,0.0,NaN,NaN,...,7,7,7,7,6,4,3,2,2,0.021048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557821,749977,09/12/20,56037,Sweetwater County,56,Wyoming,0.0,0.0,D+,F,...,7,8,7,5,3,2,1,1,1,0.025714
557822,749978,09/13/20,56037,Sweetwater County,56,Wyoming,0.0,0.0,D+,F,...,7,8,7,5,3,2,1,1,1,0.025714
557823,749979,09/14/20,56037,Sweetwater County,56,Wyoming,0.0,0.0,D+,F,...,7,8,7,5,3,2,1,1,1,0.025714
557824,749980,09/15/20,56037,Sweetwater County,56,Wyoming,0.0,0.0,D+,F,...,7,8,7,5,3,2,1,1,1,0.025714


In [62]:
# Percent % of NULL Values
null_per = new_data.isnull().mean()*100
null_per[null_per != 0]

social_distancing_total_grade              14.693829
social_distancing_visitation_grade         31.366053
social_distancing_encounters_grade         14.693829
social_distancing_travel_distance_grade    14.693829
daily_state_test                           17.560315
precipitation                              32.418173
temperature                                42.972361
ventilator_capacity_ratio                   0.299914
icu_beds_ratio                              0.299914
deaths_per_100000                           0.942588
dtype: float64

## Test-Train Split
- 80:20 Train:Test ratio
- Can't use sklearn time-series split

In [63]:
# Each county has 239 time-series instance
new_data[new_data.county_fips == 1003].shape

(239, 59)

In [64]:
# Find test train ratio
0.8*239

191.20000000000002

In [65]:
# Number of counties
new_data.county_fips.unique().shape

(2334,)

In [66]:
train = []
test = []

for i in new_data.county_fips.unique():
    train.append(new_data[new_data.county_fips == i].iloc[:191,:])
    test.append(new_data[new_data.county_fips == i].iloc[191:,:])
    
train = pd.concat(train)
test = pd.concat(test)

## Imputation 
- SimpleImputer - Categorical Columns
- KNN - Numerical Columns

In [67]:
pd.DataFrame(data.dtypes).loc[null_features[0]]

,0
social_distancing_total_grade,object
social_distancing_visitation_grade,object
social_distancing_encounters_grade,object
social_distancing_travel_distance_grade,object
daily_state_test,float64
precipitation,float64
temperature,float64
ventilator_capacity_ratio,float64
icu_beds_ratio,float64
Religious_congregation_ratio,float64


In [68]:
cat_cols = ["social_distancing_total_grade", "social_distancing_visitation_grade",
            "social_distancing_encounters_grade", "social_distancing_travel_distance_grade"]

num_cols = ["daily_state_test", "precipitation", "temperature", "ventilator_capacity_ratio",
           "icu_beds_ratio", "Religious_congregation_ratio", "percent_insured",
           "deaths_per_100000"]

In [71]:
cat_pipeline = Pipeline([("cat_imputer", SimpleImputer(missing_values = np.nan, strategy = "most_frequent"))])
num_pipeline = Pipeline([("num_imputer", KNNImputer(n_neighbors=5))])

In [72]:
full_pipeline = ColumnTransformer([
                    ("cat", cat_pipeline, cat_cols),
                    ("num", num_pipeline, num_cols)
])

In [73]:
train_imputed = full_pipeline.fit_transform(train)

In [74]:
test_imputed = full_pipeline.transform(test)

In [88]:
train_index = train.index
test_index = test.index

In [91]:
imputed_columns = ["social_distancing_total_grade", "social_distancing_visitation_grade",
            "social_distancing_encounters_grade", "social_distancing_travel_distance_grade", "daily_state_test", "precipitation", "temperature", "ventilator_capacity_ratio",
           "icu_beds_ratio", "Religious_congregation_ratio", "percent_insured",
           "deaths_per_100000"]
non_imputed_columns = [i for i in new_data.columns if i not in imputed_columns]

In [98]:
train_non_imputed = train.loc[:, non_imputed_columns]
test_non_imputed = test.loc[:, non_imputed_columns]

In [100]:
train_imputed = pd.DataFrame(train_imputed, columns = ["social_distancing_total_grade", "social_distancing_visitation_grade",
            "social_distancing_encounters_grade", "social_distancing_travel_distance_grade", "daily_state_test", "precipitation", "temperature", "ventilator_capacity_ratio",
           "icu_beds_ratio", "Religious_congregation_ratio", "percent_insured",
           "deaths_per_100000"], index = train_index).merge(
train_non_imputed, left_index=True, right_index=True)

In [101]:
test_imputed = pd.DataFrame(test_imputed, columns = ["social_distancing_total_grade", "social_distancing_visitation_grade",
            "social_distancing_encounters_grade", "social_distancing_travel_distance_grade", "daily_state_test", "precipitation", "temperature", "ventilator_capacity_ratio",
           "icu_beds_ratio", "Religious_congregation_ratio", "percent_insured",
           "deaths_per_100000"], index = test_index).merge(
test_non_imputed, left_index=True, right_index=True)

## Feature Scaling 

In [ ]:
# Something to consider!!!

# Also, lets normalize our data before knn imputation?????

## Export Imputed Data as CSV

In [105]:
with zipfile.ZipFile("/Users/Huey.ts/Desktop/STAT_390/data/imputed_data.zip", 'w', compression=zipfile.ZIP_DEFLATED) as z:
    with z.open('train.csv', 'w') as f:
        train_imputed.to_csv(f)
    with z.open('test.csv', 'w') as f:
        test_imputed.to_csv(f)